In [1]:
import feedparser
import re

feedlist = ['http://today.reuters.com/rss/topNews',
          'http://today.reuters.com/rss/domesticNews',
          'http://today.reuters.com/rss/worldNews',
          'http://hosted.ap.org/lineups/TOPHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/USHEADS-rss_2.0.xml',
          'http://rss.cnn.com/rss/edition_us.rss',
          'http://hosted.ap.org/lineups/WORLDHEADS-rss_2.0.xml',
          'http://hosted.ap.org/lineups/POLITICSHEADS-rss_2.0.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/HomePage.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/International.xml',
          'http://news.google.com/?output=rss',
          'http://feeds.salon.com/salon/news','http://www.foxnews.com/xmlfeed/rss/0,4313,0,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,80,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,81,00.rss',
          'http://www.foxnews.com/xmlfeed/rss/0,4313,0,00.rs',
          'http://rss.cnn.com/rss/edition_world.rss'
          ]

def stripHTML(h):
    p = ''
    s =0
    for c in h:
        if c=='<': s=1
        elif c=='>':
            s=0
            p+= ' '
        elif s==0: p+=c
    return p

def separatewords(text):
    splitter = re.compile('\\W*')
    return [s.lower() for s in splitter.split(text) if len(s)>3]

def getarticlewords():
    allwords = {}
    articlewords = []
    articletitles = []
    ec = 0
    
    for feed in feedlist:
        f = feedparser.parse(feed)
        print feed
        for e in f.entries:
            if e.title in articletitles: continue
            #print e.description==e.summary
            
            #if e.summary not in e: continue
            txt =e.title.encode('utf8')+stripHTML(e.description.encode('utf8'))
            words = separatewords(txt)
            
            articletitles.append(e.title)
            articlewords.append(txt)
            
            
            
            
    return articlewords,articletitles

In [2]:
artw,artt = getarticlewords()

http://today.reuters.com/rss/topNews
http://today.reuters.com/rss/domesticNews
http://today.reuters.com/rss/worldNews
http://hosted.ap.org/lineups/TOPHEADS-rss_2.0.xml
http://hosted.ap.org/lineups/USHEADS-rss_2.0.xml
http://rss.cnn.com/rss/edition_us.rss
http://hosted.ap.org/lineups/WORLDHEADS-rss_2.0.xml
http://hosted.ap.org/lineups/POLITICSHEADS-rss_2.0.xml
http://www.nytimes.com/services/xml/rss/nyt/HomePage.xml
http://www.nytimes.com/services/xml/rss/nyt/International.xml
http://news.google.com/?output=rss
http://feeds.salon.com/salon/news
http://www.foxnews.com/xmlfeed/rss/0,4313,0,00.rss
http://www.foxnews.com/xmlfeed/rss/0,4313,80,00.rss
http://www.foxnews.com/xmlfeed/rss/0,4313,81,00.rss
http://www.foxnews.com/xmlfeed/rss/0,4313,0,00.rs
http://rss.cnn.com/rss/edition_world.rss


In [3]:
print artw[:10]

["Police: Maryland school shooter was killed by his own bullet, not the school resource officer'sAlthough a school resource officer did fire at the teen who'd shot two fellow students at a Maryland high school last week, the shooter died of a bullet from his own gun, authorities say. ", "There has been, on average, 1 school shooting every week this yearWe're only 12 weeks into 2018, and there have already been 17 school shootings where someone was hurt or killed. That averages out to 1.4 shootings a week. ", 'Girl critically wounded in Maryland school shooting diesThe 16-year-old girl wounded by an armed student at Great Mills High School in southern Maryland early this week has died, her family and authorities said Friday. ', 'Maryland governor: We need more than prayersMaryland Governor Larry Hogan remarks that more than thoughts and prayers are needed after a school shooting at Great Mills High School in Maryland. ', "CNN Poll: Nearly 2/3 approve of Trump's plan to meet Kim Jong UnW

In [4]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

vectorizer = CountVectorizer(min_df=3,max_df=0.95,stop_words="english")
x = vectorizer.fit_transform(artw)
nmf = NMF(n_components=20,random_state=1,
          alpha=.1, l1_ratio=.5)
w  = nmf.fit_transform(x)
h = nmf.components_


In [55]:
from numpy import *
from operator import itemgetter, attrgetter, methodcaller
def showfeatures(w,h,titles,wordvec,out="features.txt"):
    
    outfile = file(out,'w')
    pc,wc = shape(h)
    toppatterns=[[] for i in range(len(titles))]
    
    patternnames = []
    
    #loop for each features
    
    for i in range(pc):
        slist = []
        
        for j in range(wc):
            slist.append((h[i,j],wordvec[j]))
        a=sorted(slist,key=itemgetter(0),reverse=True)
        
        
        n = [s[1] for s in a[0:6]]
        outfile.write(str(n)+'\n')
        patternnames.append(n)
        
        flist = []
        for j in range(len(titles)):
            flist.append((w[j,i],titles[j]))
            toppatterns[j].append((w[j,i],i,titles))
            
        flist.sort()
        flist.reverse()
        
        for f in flist[0:3]:
            outfile.write(str(f)+'\n')
            outfile.write('\n')
            
    outfile.close()
    return toppatterns,patternnames
    

In [52]:
wordvec = vectorizer.get_feature_names()
topp,pn = showfeatures(w,h,artt,wordvec)
#pc,wc= shape(h)

#for i in range(pc):
#    slist = []
#    for j in range(wc):
#        slist.append((h[i,j],x[j]))
    


In [53]:
def showarticles(titles,toppatterns,patternnames,out='articles.txt'):
    
    outfile = file(out,'w')
    
    for j in range(len(titles)):
        outfile.write(titles[j].encode('utf8')+'\n')
        
        toppatterns[j].sort()
        toppatterns[j].reverse()
        
        for i in range(3):
            outfile.write(str(toppatterns[j][i][0])+' '+
                         str(patternnames[toppatterns[j][i][1]])+'\n')
            
        outfile.write('\n')
        
    outfile.close()

In [54]:
showarticles(artt,topp,pn)